# Experimentos de Ingeniería de Prompts con Datos de Alimentos

Este notebook permite realizar experimentos de ingeniería de prompts utilizando un dataset de alimentos y la API de OpenAI para explorar mapeos entre sabores, emociones y características físicas.

## 1. Import Required Libraries

Importamos las librerías necesarias para manipulación de datos e interacción con la API de OpenAI.

In [1]:
import pandas as pd
from openai import OpenAI
import os
import time
from typing import List, Dict, Any
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Importar el módulo de configuración
from config.config import load_config, setup_project_paths, PROJECT_ROOT

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 2. Load and Explore Dataset

Cargamos el archivo `food_curated_sample.csv` y exploramos su estructura.

In [2]:
# Configurar directorios del proyecto y cargar configuración
setup_project_paths()  # Crea todos los directorios necesarios
config = load_config()

In [3]:
# Cargar el dataset de alimentos
df = pd.read_csv(f'{PROJECT_ROOT}/{config.data.cleaned_data_path}/food/food_curated_sample.csv', header=None, names=['food_item'])

print(f"Dataset cargado exitosamente!")
print(f"Número total de elementos: {len(df)}")
print(f"Primeros 10 elementos:")
print(df.head(10))
print(f"\nÚltimos 5 elementos:")
print(df.tail(5))

Dataset cargado exitosamente!
Número total de elementos: 160
Primeros 10 elementos:
   food_item
0  Chocolate
1  Chocolate
2  Chocolate
3  Chocolate
4  Chocolate
5  Chocolate
6  Chocolate
7  Chocolate
8  Chocolate
9  Chocolate

Últimos 5 elementos:
                    food_item
155            dulce de leche
156             mint lemonade
157  banana and kiwi smoothie
158                    whisky
159               panna cotta


## 3. Define Prompt Templates

Definimos plantillas de prompts que permiten reemplazar partes del texto con valores del dataset.

In [ ]:
# Definir plantillas de prompts
prompt_templates = {
# Plantilla 1: Mapeo de sensaciones y emociones
    "sensations_emotions_mapping": """Analyze the flavor profile of the following food item: {food_item}
Provide:
1. sensations (chemical flavor profile)
2. human responses.
3. temperature.
4. texture.
5. emotions (5 basic emotions) anger, disgust, fear, happiness, sadness, and surprise.
6. color
7. taste (sweet, bitter, salty, sour).
Format your response as: sensations|human_responses|temperature|texture|emotions|color|taste""",
# Plantilla 2: Mapeo de sensaciones y emociones (variante)
    "sensations_emotions_mapping_2": """Analyze the flavor profile of the following food item: {food_item}
Reflect over the casual relationship of the sensations the food term produces to human experience and provide the top 3 aspects for each category in a coma-separated format:
1. sensations (chemical flavor profile)
2. human responses.
3. temperature.
4. texture.
5. emotions (5 basic emotions) anger, disgust, fear, happiness, sadness, and surprise.
6. color
7. taste (sweet, bitter, salty, sour).
Format your response as: sensations|human_responses|temperature|texture|emotions|color|taste""",
# Plantilla 3: Mapeo de sensaciones y emociones (variante)
    "sensations_emotions_mapping_3": """Analyze the flavor profile of the following food item: {food_item}
Reflect over the casual relationship of the sensations the food term produces to human experience and provide the top 3 aspects for each category, and reduce the aspects to single terms in a coma-separated format:
1. sensations (chemical flavor profile)
2. human responses.
3. temperature.
4. texture.
5. emotions. Narrow to the 5 basic emotions: (anger, disgust, fear, happiness, sadness, and surprise).
6. color
7. taste. Narrow to the basic tastes: (sweet, bitter, salty, sour).
Format your response as: sensations|human_responses|temperature|texture|emotions|color|taste""",

    "sensations_emotions_mapping_no_bias": """
        Survey on associations between sensations, emotions, and tastes of `{}`. Answer briefly:

        1. Chemical flavor profile - top 3 sensation terms
        2. Human responses - top 3 physiological effects
        3. Temperature - choose: hot, warm, cold, iced
        4. Texture - top 3 texture terms
        5. Emotions - choose: anger, disgust, fear, happiness, sadness, surprise
        6. Color - choose: black, blue, brown, green, gray, orange, pink, purple, red, white, yellow
        7. Taste - choose: sweet, bitter, salty, sour

        Respond with single terms, comma-separated, formatted as:
        Chemical flavor profile|Human responses|Temperature|Texture|Emotions|Color|Taste
        In case the `food_item` is not a food returns only `No Label`. If any dimension cannot answered, use the label `No Label`
    """
# Plantilla 3: Mapeo de sensaciones y emociones (variante tipo encuesta)
    "sensations_emotions_mapping_4": """This is a survey on associations between sensations, emotions, and tastes. I would like to inquire about the sensations the {food_item} produces in you by answering the following questions:

1.  Chemical flavor profile: What are the top 3 terms that better describe the sensations related to the {food_item} chemical profile?
2. Human responses: What are the top 3 terms that better describe the physiological effects related to eating {food_item}?
3. Temperature:  Among the following temperature terms (hot, warm, cold, iced), which term do you think goes best with {food_item}?
4. Texture: What are the top 3 terms that better describe the texture of {food_item}?
5. Emotions: Among the following emotions (anger, disgust, fear, happiness, sadness, and surprise), which emotion do you think goes best with {food_item}?
6. Color: Among the following colors (black, blue, brown, green, gray, orange, pink, purple, red,  white, yellow), which color do you think goes best with {food_item}?
7. Taste: Among the following basic tastes (sweet, bitter, salty, sour), which taste do you think goes best with {food_item}?

For each question, please provide up to 3 terms.

Reduce your answers to single terms in a coma-separated list, and format your response as follows: `Chemical flavor profile` | `Human responses` | `Temperature` | `Texture` | `Emotions` | `Color` | `Taste`""",
# Plantilla 5: Mapeo de sensaciones y emociones (variante tipo encuesta reducida en tokens)
    "sensations_emotions_mapping_5": """Survey on associations between sensations, emotions, and tastes of `{food_item}`. Answer briefly:

1. Chemical flavor profile - top 3 sensation terms  
2. Human responses - top 3 physiological effects  
3. Temperature - choose: hot, warm, cold, iced  
4. Texture - top 3 texture terms  
5. Emotions - choose: anger, disgust, fear, happiness, sadness, surprise  
6. Color - choose: black, blue, brown, green, gray, orange, pink, purple, red, white, yellow  
7. Taste - choose: sweet, bitter, salty, sour  

Respond with single terms, comma-separated, formatted as:  
`Chemical flavor profile` | `Human responses` | `Temperature` | `Texture` | `Emotions` | `Color` | `Taste`""",
# Plantilla 6: Mapeo de sensaciones y emociones (variante tipo encuesta sin sesgo)
    "sensations_emotions_mapping_no_bias": """
        Survey on associations between sensations, emotions, and tastes of `{food_item}`. Answer briefly:

        1. Chemical flavor profile - top 3 sensation terms
        2. Human responses - top 3 physiological effects
        3. Temperature - choose: hot, warm, cold, iced
        4. Texture - top 3 texture terms
        5. Emotions - choose: anger, disgust, fear, happiness, sadness, surprise
        6. Color - choose: black, blue, brown, green, gray, orange, pink, purple, red, white, yellow
        7. Taste - choose: sweet, bitter, salty, sour

        Respond with single terms, comma-separated, formatted as:
        Chemical flavor profile|Human responses|Temperature|Texture|Emotions|Color|Taste
        In case the `food_item` is not a food returns only `No Label`. If any dimension cannot answered, use the label `No Label`
    """
}

print(f"Definidas {len(prompt_templates)} plantillas de prompts:")
for name, template in prompt_templates.items():
    print(f"- {name}")
    print(f"  Longitud: {len(template)} caracteres")
    print()

Definidas 4 plantillas de prompts:
Definidas 4 plantillas de prompts:
- sensations_emotions_mapping
  Longitud: 391 caracteres

- sensations_emotions_mapping_2
  Longitud: 556 caracteres

- sensations_emotions_mapping_3
  Longitud: 641 caracteres

- sensations_emotions_mapping_4
  Longitud: 1372 caracteres

- sensations_emotions_mapping_no_bias
  Longitud: 837 caracteres



## 4. Configure OpenAI API

Configuramos las credenciales y parámetros de conexión para la API de OpenAI.

In [ ]:
# Configurar OpenAI API
def setup_openai_client():
    """
    Configura el cliente de OpenAI usando la variable de entorno OPENAI_API_KEY
    """
    try:
        # Intentar obtener la API key de la variable de entorno
        api_key = os.getenv('OPENAI_API_KEY')
        
        if not api_key:
            print("OPENAI_API_KEY no encontrada en variables de entorno")
            print("Por favor, configura tu API key ejecutando:")
            print("export OPENAI_API_KEY='tu-api-key-aquí'")
            return None
            
        # Configurar el cliente OpenAI (nueva API v1.0+)
        client = OpenAI(api_key=api_key)
        print("✅ Cliente OpenAI configurado correctamente")
        return client
        
    except Exception as e:
        print(f"❌ Error configurando OpenAI: {e}")
        return None

# Configuración de parámetros del modelo
MODEL_CONFIG = {
    "model": "gpt-4o-mini",  # Actualizado a un modelo válido
    # "model": "gpt-5-nano",
    # "temperature": 0.2,
    # "max_tokens": 300,
    # "top_p": 1.0,
    # "frequency_penalty": 0.0,
    # "presence_penalty": 0.0
}

print("Configuración del modelo:")
for key, value in MODEL_CONFIG.items():
    print(f"  {key}: {value}")

# Configurar cliente y guardarlo en variable global
openai_client = setup_openai_client()

Configuración del modelo:
  model: gpt-4o-mini
✅ Cliente OpenAI configurado correctamente


## 5. Create Prompt Processing Functions

Definimos funciones para formatear prompts, realizar llamadas a la API y manejar respuestas.

In [6]:
def format_prompt(template: str, food_item: str) -> str:
    """
    Formatea una plantilla de prompt reemplazando el placeholder con el item de comida
    
    Args:
        template: Plantilla de prompt con placeholder {food_item}
        food_item: Item de comida para insertar
    
    Returns:
        Prompt formateado
    """
    return template.format(food_item=food_item.strip().lower())

def call_openai_api(prompt: str, retries: int = 3, delay: float = 1.0) -> Dict[str, Any]:
    """
    Realiza una llamada a la API de OpenAI con manejo de errores y reintentos
    
    Args:
        prompt: Prompt a enviar
        retries: Número de reintentos en caso de error
        delay: Retraso entre reintentos
    
    Returns:
        Diccionario con respuesta y metadatos
    """
    if not openai_client:
        return {
            "success": False,
            "response": None,
            "error": "Cliente OpenAI no configurado",
            "attempt": 0
        }
    
    for attempt in range(retries):
        try:
            # Usar la nueva API v1.0+ con el cliente configurado
            response = openai_client.chat.completions.create(
                model=MODEL_CONFIG["model"],
                messages=[
                    {"role": "user", "content": prompt}
                ],
                # temperature=MODEL_CONFIG["temperature"],
                # max_tokens=MODEL_CONFIG["max_tokens"],
                # top_p=MODEL_CONFIG["top_p"],
                # frequency_penalty=MODEL_CONFIG["frequency_penalty"],
                # presence_penalty=MODEL_CONFIG["presence_penalty"]
            )
            
            return {
                "success": True,
                "response": response.choices[0].message.content.strip(),
                "usage": response.usage.model_dump(),  # Convertir a dict para compatibilidad
                "model": response.model,
                "attempt": attempt + 1
            }
            
        except Exception as e:
            error_type = type(e).__name__
            print(f"❌ Error {error_type} (intento {attempt + 1}): {str(e)}")
            
            # Manejo específico de errores de rate limiting
            if "rate_limit" in str(e).lower() or "429" in str(e):
                wait_time = delay * (attempt + 1) * 2  # Incrementar tiempo de espera
                print(f"⏱️  Rate limit detectado. Esperando {wait_time:.1f} segundos...")
                time.sleep(wait_time)
            elif attempt < retries - 1:
                time.sleep(delay)
    
    return {
        "success": False,
        "response": None,
        "error": "Falló después de todos los reintentos",
        "attempt": retries
    }

def process_single_experiment(food_item: str, template_name: str, template: str) -> Dict[str, Any]:
    """
    Procesa un experimento individual: formatea prompt, llama API y registra resultado
    
    Args:
        food_item: Item de comida
        template_name: Nombre de la plantilla
        template: Plantilla de prompt
    
    Returns:
        Diccionario con resultados del experimento
    """
    # Formatear prompt
    formatted_prompt = format_prompt(template, food_item)
    
    # Llamar API
    api_result = call_openai_api(formatted_prompt)
    
    # Estructurar resultado
    result = {
        "food_item": food_item,
        "template_name": template_name,
        "prompt": formatted_prompt,
        "response": api_result.get("response"),
        "success": api_result["success"],
        "tokens_used": api_result.get("usage", {food_item}).get("total_tokens", 0) if api_result["success"] else 0,
        "in_tokens": api_result.get("usage", {}).get("prompt_tokens") if api_result["success"] else 0,
        "out_tokens": api_result.get("usage", {}).get("completion_tokens") if api_result["success"] else 0,
        "model_used": api_result.get("model", MODEL_CONFIG["model"]),
        "timestamp": pd.Timestamp.now(),
        "attempt": api_result.get("attempt", 0)
    }
    
    if not api_result["success"]:
        result["error"] = api_result.get("error")
    
    return result

print("✅ Funciones de procesamiento de prompts definidas correctamente")

✅ Funciones de procesamiento de prompts definidas correctamente


## 6. Execute Prompt Experiments

Ejecutamos los experimentos iterando a través de las plantillas de prompts y elementos del dataset.

In [7]:
def run_experiments(food_items: List[str], 
                   templates: Dict[str, str], 
                   max_items: int = None,
                   selected_templates: List[str] = None) -> List[Dict[str, Any]]:
    """
    Ejecuta experimentos de prompts para una lista de elementos de comida
    
    Args:
        food_items: Lista de elementos de comida
        templates: Diccionario de plantillas de prompts
        max_items: Máximo número de elementos a procesar (None = todos)
        selected_templates: Lista de nombres de plantillas a usar (None = todas)
    
    Returns:
        Lista de resultados de experimentos
    """
    # Filtrar plantillas si se especifica
    if selected_templates:
        templates = {k: v for k, v in templates.items() if k in selected_templates}
    
    # Limitar elementos si se especifica
    if max_items:
        food_items = food_items[:max_items]
    
    results = []
    total_experiments = len(food_items) * len(templates)

    print(f"Iniciando {total_experiments} experimentos...")
    print(f"   - {len(food_items)} elementos de comida")
    print(f"   - {len(templates)} plantillas de prompts")
    print(f"   - Plantillas: {list(templates.keys())}")
    print()
    
    # Crear barra de progreso
    with tqdm(total=total_experiments, desc="Experimentos") as pbar:
        for food_item in food_items:
            for template_name, template in templates.items():
                try:
                    # Procesar experimento individual
                    result = process_single_experiment(food_item, template_name, template)
                    results.append(result)
                    
                    # Actualizar barra de progreso
                    status = "OK" if result["success"] else "Fallido"
                    pbar.set_postfix({
                        'Item': food_item[:20] + "..." if len(food_item) > 20 else food_item,
                        'Template': template_name[:15] + "..." if len(template_name) > 15 else template_name,
                        'Status': status
                    })
                    pbar.update(1)
                    
                    # Pequeña pausa para evitar rate limiting
                    time.sleep(0.25)
                    
                except Exception as e:
                    print(f"Error procesando {food_item} con {template_name}: {e}")
                    pbar.update(1)
    
    # Estadísticas finales
    successful = sum(1 for r in results if r["success"])
    failed = len(results) - successful
    total_tokens = sum(r["tokens_used"] for r in results)
    
    print(f"\nResumen de experimentos:")
    print(f"   - Total: {len(results)}")
    print(f"   - Exitosos: {successful}")
    print(f"   - Fallidos: {failed}")
    print(f"   - Tokens usados: {total_tokens}")
    print(f"   - Tasa de éxito: {successful/len(results)*100:.1f}%")
    
    return results

In [8]:

# Configuración de experimento
EXPERIMENT_CONFIG = {
    "max_items": None,  # Cambiar a None para procesar todos los elementos
    # "selected_templates": ["sensations_emotions_mapping","sensations_emotions_mapping_2","sensations_emotions_mapping_3"],  # None para usar todas las plantillas
    "selected_templates": ["sensations_emotions_mapping_no_bias", ],
    "save_results": True
}

print("⚙️  Configuración del experimento:")
for key, value in EXPERIMENT_CONFIG.items():
    print(f"   {key}: {value}")
print()

⚙️  Configuración del experimento:
   max_items: None
   selected_templates: ['sensations_emotions_mapping_no_bias']
   save_results: True



In [9]:
# Ejecutar experimentos
if openai_client:
    # Obtener lista de elementos de comida que se encuentran en la primera columna del dataset
    food_list = df.iloc[:, 0].tolist()
    
    # Ejecutar experimentos
    experiment_results = run_experiments(
        food_items=food_list,
        templates=prompt_templates,
        max_items=EXPERIMENT_CONFIG["max_items"],
        selected_templates=EXPERIMENT_CONFIG["selected_templates"]
    )
else:
    print("⚠️  No se puede ejecutar experimentos sin cliente OpenAI configurado")
    print("Configurando resultados de ejemplo...")
    
    # Crear resultados de ejemplo para demostración
    experiment_results = [
        {
            "food_item": "kadhi",
            "template_name": "sensations_emotions_mapping", 
            "prompt": "I will give you a food item name...",
            "response": "tangy, creamy, warming | comfort, nostalgia, satisfaction | warm, medium heat | smooth, thick, velvety",
            "success": True,
            "tokens_used": 45,
            "model_used": "gpt-4o-mini",
            "timestamp": pd.Timestamp.now(),
            "attempt": 1
        }
    ]
    print("✅ Resultados de ejemplo creados")

Iniciando 10 experimentos...
   - 10 elementos de comida
   - 1 plantillas de prompts
   - Plantillas: ['sensations_emotions_mapping_no_bias']



Experimentos: 100%|██████████| 70/70 [01:47<00:00,  1.54s/it, Item=panna cotta, Template=sensations_emot..., Status=OK]            


Resumen de experimentos:
   - Total: 10
   - Exitosos: 10
   - Fallidos: 0
   - Tokens usados: 17113
   - Tasa de éxito: 100.0%


## 7. Save Results to DataFrame

Almacenamos los resultados de los experimentos en un DataFrame estructurado para análisis posterior.

In [11]:
# Crear DataFrame con resultados
results_df = pd.DataFrame(experiment_results)

print("📊 DataFrame de resultados creado:")
print(f"Shape: {results_df.shape}")
print(f"Columnas: {list(results_df.columns)}")
print()

# Mostrar muestra de resultados
if len(results_df) > 0:
    print("🔍 Muestra de resultados:")
    display_columns = ['food_item', 'template_name', 'success', 'in_tokens', 'out_tokens']
    print(results_df[display_columns].head())
    
    print("\n📝 Ejemplo de respuesta completa:")
    first_successful = results_df[results_df['success'] == True].iloc[0]
    print(f"Elemento: {first_successful['food_item']}")
    print(f"Plantilla: {first_successful['template_name']}")
    print(f"Respuesta: {first_successful['response']}")
    print(f"Tokens: {first_successful['tokens_used']}")

# Guardar resultados si está configurado
if EXPERIMENT_CONFIG.get("save_results", False) and len(results_df) > 0:
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    filename = f"food/prompt_experiment_results_{timestamp}.csv"
    results_df.to_csv(PROJECT_ROOT / config.data.cleaned_data_path / filename, index=False, columns=['food_item', 'template_name', 'response', 'success', 'model_used'])
    print(f"\n💾 Resultados guardados en: {filename}")

print("\n✅ Experimentos completados exitosamente!")

📊 DataFrame de resultados creado:
Shape: (70, 11)
Columnas: ['food_item', 'template_name', 'prompt', 'response', 'success', 'tokens_used', 'in_tokens', 'out_tokens', 'model_used', 'timestamp', 'attempt']

🔍 Muestra de resultados:
     food_item                        template_name  success  in_tokens  \
0    Chocolate  sensations_emotions_mapping_no_bias     True        212   
1  creamy peas  sensations_emotions_mapping_no_bias     True        213   
2       aperol  sensations_emotions_mapping_no_bias     True        212   
3    croissant  sensations_emotions_mapping_no_bias     True        212   
4       Yogurt  sensations_emotions_mapping_no_bias     True        213   

   out_tokens  
0          31  
1          29  
2          34  
3          31  
4          31  

📝 Ejemplo de respuesta completa:
Elemento: Chocolate
Plantilla: sensations_emotions_mapping_no_bias
Respuesta: chocolatey, rich, creamy|pleasure, satisfaction, energy|warm|smooth, velvety, crunchy|happiness|brown|sweet
Tok

In [11]:
# Crear DataFrame con resultados
results_df = pd.DataFrame(experiment_results)

print("📊 DataFrame de resultados creado:")
print(f"Shape: {results_df.shape}")
print(f"Columnas: {list(results_df.columns)}")
print()

# Mostrar muestra de resultados
if len(results_df) > 0:
    print("🔍 Muestra de resultados:")
    display_columns = ['food_item', 'template_name', 'success', 'in_tokens', 'out_tokens']
    print(results_df[display_columns].head())
    
    print("\n📝 Ejemplo de respuesta completa:")
    first_successful = results_df[results_df['success'] == True].iloc[0]
    print(f"Elemento: {first_successful['food_item']}")
    print(f"Plantilla: {first_successful['template_name']}")
    print(f"Respuesta: {first_successful['response']}")
    print(f"Tokens: {first_successful['tokens_used']}")

# Guardar resultados si está configurado
if EXPERIMENT_CONFIG.get("save_results", False) and len(results_df) > 0:
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    filename = f"food/prompt_experiment_results_{timestamp}.csv"
    results_df.to_csv(PROJECT_ROOT / config.data.cleaned_data_path / filename, index=False, columns=['food_item', 'template_name', 'response', 'success', 'model_used', 'in_tokens', 'out_tokens'])
    print(f"\n💾 Resultados guardados en: {filename}")

print("\n✅ Experimentos completados exitosamente!")

📊 DataFrame de resultados creado:
Shape: (70, 11)
Columnas: ['food_item', 'template_name', 'prompt', 'response', 'success', 'tokens_used', 'in_tokens', 'out_tokens', 'model_used', 'timestamp', 'attempt']

🔍 Muestra de resultados:
     food_item                        template_name  success  in_tokens  \
0    Chocolate  sensations_emotions_mapping_no_bias     True        212   
1  creamy peas  sensations_emotions_mapping_no_bias     True        213   
2       aperol  sensations_emotions_mapping_no_bias     True        212   
3    croissant  sensations_emotions_mapping_no_bias     True        212   
4       Yogurt  sensations_emotions_mapping_no_bias     True        213   

   out_tokens  
0          31  
1          29  
2          34  
3          31  
4          31  

📝 Ejemplo de respuesta completa:
Elemento: Chocolate
Plantilla: sensations_emotions_mapping_no_bias
Respuesta: chocolatey, rich, creamy|pleasure, satisfaction, energy|warm|smooth, velvety, crunchy|happiness|brown|sweet
Tok

In [12]:
experiment_results[:5]

[{'food_item': 'Chocolate',
  'template_name': 'sensations_emotions_mapping_no_bias',
  'prompt': '\n        Survey on associations between sensations, emotions, and tastes of `chocolate`. Answer briefly:\n\n        1. Chemical flavor profile - top 3 sensation terms\n        2. Human responses - top 3 physiological effects\n        3. Temperature - choose: hot, warm, cold, iced\n        4. Texture - top 3 texture terms\n        5. Emotions - choose: anger, disgust, fear, happiness, sadness, surprise\n        6. Color - choose: black, blue, brown, green, gray, orange, pink, purple, red, white, yellow\n        7. Taste - choose: sweet, bitter, salty, sour\n\n        Respond with single terms, comma-separated, formatted as:\n        Chemical flavor profile|Human responses|Temperature|Texture|Emotions|Color|Taste\n        In case the `food_item` is not a food returns only `No Label`. If any dimension cannot answered, use the label `No Label`\n    ',
  'response': 'chocolatey, rich, creamy|